<a href="https://colab.research.google.com/github/Saad-cpp/Data-Science-Internship/blob/main/Basic_GraphRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://token@github.com/neo4j-graphacademy/llm-vectors-unstructured

Cloning into 'llm-vectors-unstructured'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 178 (delta 29), reused 80 (delta 17), pack-reused 69 (from 1)
Receiving objects: 100% (178/178), 47.99 MiB | 18.14 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [ ]:
!pip install -r /content/llm-vectors-unstructured/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49

In [ ]:
!pip install google-generativeai

In [ ]:
from google.colab import userdata
gem_key = userdata.get('GEMINI-KEY')
neo_url = userdata.get('neo-url')
neo_user = userdata.get('neo-user')
neo_pass = userdata.get('neo-pass')

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
import google.generativeai as genai
from neo4j import GraphDatabase
from langchain_community.vectorstores.neo4j_vector import Neo4jVector

genai.configure(api_key=gem_key)

COURSES_PATH = "/content/llm-vectors-unstructured/llm-vectors-unstructured/data/asciidoc"

loader = DirectoryLoader(COURSES_PATH, glob="**/lesson.adoc", loader_cls=TextLoader)
docs = loader.load()

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1500,
    chunk_overlap=200,
)

chunks = text_splitter.split_documents(docs)


In [ ]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.6
    Uninstalling google-ai-generativelanguage-0.6.6:
      Successfully uninstalled google-ai-generativelanguage-0.6.6
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.7.2
    Uninstalling google-generativeai-0.7.2:
      Successfully uninstalled google-generativeai-0.7.2


# **CREATING VECTOR FROM DOCUMENTS**

In [ ]:
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain_google_genai import GoogleGenerativeAIEmbeddings

neo4j_db = Neo4jVector.from_documents(
    chunks,
    GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", google_api_key = gem_key),
    url=neo_url,
    username=neo_user,
    password=neo_pass,
    database="neo4j",
    index_name="chunkVector",
    node_label="Chunk",
    text_node_property="text",
    embedding_node_property="embedding",
)


In [ ]:

response = genai.embed_content(model="models/text-embedding-004", content="What does hallucination mean?")


[0.0119056925, 0.0048198216, -0.03259242, -0.044548422, -0.07073696, 0.022851354, 0.006630087, -0.02595581, -0.08095177, -0.015849285, 0.033063997, -0.0042900923, 0.0028481234, -0.02006189, -0.04180651, -0.077049285, 0.023721687, 0.062067565, -0.03783016, -0.009402408, -0.015281742, -0.028301034, 0.0073960163, -0.09713471, -0.0224557, 0.0009633503, 0.029324098, 0.026870906, -0.036750205, 0.042385515, 0.05897531, -0.010236744, 0.072089665, 0.04424017, 0.04958447, 0.044356085, 0.011106528, 0.04393049, -0.000107527114, -0.064931884, -0.038579054, 0.013928379, -0.05128461, 0.037999574, -0.071024135, 0.08157883, 0.015119018, 0.06429069, 0.009956238, -0.0076128882, -0.009618708, 0.027581543, -0.039176147, 0.07421379, -0.03317814, -0.0030371428, -0.026351376, -0.027670817, 0.001293356, 0.0014247712, -0.05697347, 0.015356495, -0.027492633, 1.1410143e-05, -0.04104694, -0.013220434, -0.052097112, -0.036123518, -0.049761225, 0.038424842, -0.041618172, 0.051542133, -0.057390854, 0.0073156497, -0.0

In [ ]:
print(response['embedding'])
embedding = response['embedding']

[0.0119056925, 0.0048198216, -0.03259242, -0.044548422, -0.07073696, 0.022851354, 0.006630087, -0.02595581, -0.08095177, -0.015849285, 0.033063997, -0.0042900923, 0.0028481234, -0.02006189, -0.04180651, -0.077049285, 0.023721687, 0.062067565, -0.03783016, -0.009402408, -0.015281742, -0.028301034, 0.0073960163, -0.09713471, -0.0224557, 0.0009633503, 0.029324098, 0.026870906, -0.036750205, 0.042385515, 0.05897531, -0.010236744, 0.072089665, 0.04424017, 0.04958447, 0.044356085, 0.011106528, 0.04393049, -0.000107527114, -0.064931884, -0.038579054, 0.013928379, -0.05128461, 0.037999574, -0.071024135, 0.08157883, 0.015119018, 0.06429069, 0.009956238, -0.0076128882, -0.009618708, 0.027581543, -0.039176147, 0.07421379, -0.03317814, -0.0030371428, -0.026351376, -0.027670817, 0.001293356, 0.0014247712, -0.05697347, 0.015356495, -0.027492633, 1.1410143e-05, -0.04104694, -0.013220434, -0.052097112, -0.036123518, -0.049761225, 0.038424842, -0.041618172, 0.051542133, -0.057390854, 0.0073156497, -0.0

In [ ]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=neo_url,
    username=neo_user,
    password=neo_pass,
)

In [ ]:
result = graph.query("""
CALL db.index.vector.queryNodes('chunkVector', 6, $embedding)
YIELD node, score
RETURN node.text, score
""", {"embedding": embedding})


In [ ]:
for row in result:
    print(row['node.text'], row['score'])

= Avoiding Hallucination
:order: 2
:type: lesson

As you learned in the previous lesson, LLMs can "make things up".

LLMs are designed to generate human-like text based on the patterns they've identified in vast amounts of data. 

Due to their reliance on patterns and the sheer volume of training information, LLMs sometimes **hallucinate** or produce outputs that manifest as generating untrue facts, asserting details with unwarranted confidence, or crafting plausible yet nonsensical explanations.

These manifestations arise from a mix of _overfitting_, biases in the training data, and the model's attempt to generalize from vast amounts of information.

== Common Hallucination Problems

Let's take a closer look at some reasons why this may occur.

=== Temperature

LLMs have a _temperature_, corresponding to the amount of randomness the underlying model should use when generating the text.

The higher the temperature value, the more random the generated result will become, and the more l

In [ ]:
print(len(chunks))

89


# **CREATING MY OWN EMBEDDINGS**

In [ ]:
!python -m textblob.download_corpora

In [ ]:
from textblob import TextBlob

phrase = "You can extract topics from phrases using TextBlob"

topics = TextBlob(phrase).noun_phrases

print(topics)

In [ ]:
def get_embedding(llm, text):
    response = llm.embed_content(
            content=text,
            model="models/text-embedding-004"
        )
    return response["embedding"]

In [ ]:
import os

def get_course_data(llm, chunk):
    data = {}

    path = chunk.metadata['source'].split(os.path.sep)

    data['course'] = path[-6]
    data['module'] = path[-4]
    data['lesson'] = path[-2]
    data['url'] = f"https://graphacademy.neo4j.com/courses/{data['course']}/{data['module']}/{data['lesson']}"
    data['text'] = chunk.page_content
    data['embedding'] = get_embedding(llm, data['text'])
    data['topics'] = TextBlob(data['text']).noun_phrases

    return data

In [ ]:
driver = GraphDatabase.driver(
    neo_url,
    auth=(
        neo_user,
        neo_pass
    )
)
driver.verify_connectivity()

In [ ]:
def create_chunk(tx, data):
    tx.run("""
        MERGE (c:Course {name: $course})
        MERGE (c)-[:HAS_MODULE]->(m:Module{name: $module})
        MERGE (m)-[:HAS_LESSON]->(l:Lesson{name: $lesson, url: $url})
        MERGE (l)-[:CONTAINS]->(p:Paragraph{text: $text})
        WITH p
        CALL db.create.setNodeVectorProperty(p, "embedding", $embedding)
        FOREACH ( topic in $topics |
        MERGE (p)-[:MENTIONS]->(t:Topic{name: topic})
        )
        """,
        data
        )

In [ ]:
llm = genai
for chunk in chunks:
    with driver.session(database="neo4j") as session:

        session.execute_write(
            create_chunk,
            get_course_data(llm, chunk)
        )

In [ ]:
with driver.session(database="neo4j") as session:
    session.run("""
        CREATE VECTOR INDEX paragraphs IF NOT EXISTS
        FOR (p:Paragraph) ON (p.embedding)
        OPTIONS { indexConfig: {
         `vector.dimensions`: 768,
          `vector.similarity_function`: 'cosine'
        }}
    """)

In [ ]:
driver.close()

In [ ]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain_google_genai import GoogleGenerativeAIEmbeddings

my_vector = Neo4jVector.from_existing_index(
    GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", google_api_key = gem_key),
    url=neo_url,
    username=neo_user,
    password=neo_pass,
    node_label='Paragraph',
    index_name = 'paragraphs',
    embedding_node_property="embedding",
    text_node_property="text",
)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=gem_key)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=my_vector.as_retriever(),
    verbose=True,
    return_source_documents=True)

In [ ]:
response = qa_chain.invoke({'query':"What does hallucination mean?"})
print("Query: ", response['query'])
print('\n\n')
print("Response:", response['result'])
print('\n\n')
print("="*200)
print("Sources: ")
last_page = ""
for doc in response['source_documents']:
    print(doc.page_content)
    print("="*200)
    print('\n\n')
    last_page = doc.page_content



> Entering new RetrievalQA chain...

> Finished chain.
Query:  What does hallucination mean?



Response: In the context of Large Language Models (LLMs), **hallucination** refers to the phenomenon where the model generates outputs that are factually incorrect, lack logical coherence, or are based on biases in the training data. 

It's like the model is "making things up" or creating information that doesn't exist in reality. 




Sources: 
= Avoiding Hallucination
:order: 2
:type: lesson

As you learned in the previous lesson, LLMs can "make things up".

LLMs are designed to generate human-like text based on the patterns they've identified in vast amounts of data. 

Due to their reliance on patterns and the sheer volume of training information, LLMs sometimes **hallucinate** or produce outputs that manifest as generating untrue facts, asserting details with unwarranted confidence, or crafting plausible yet nonsensical explanations.

These manifestations arise from a mix of _overfitti

In [ ]:
print(last_page)

= Grounding LLMs
:order: 3
:type: lesson

In the previous lesson, you learned about the potential methods that you could use to avoid _Hallucination_, including _Grounding_.

In this lesson, you will explore grounding and an approach known as _Retrieval Augmented Generation_ (RAG).

[.video]
video::Le_j18w2oGI[youtube,width=560,height=315]

[.transcript]
== Grounding LLMs with RAG

Grounding is the process of providing context to an LLM to improve the accuracy of its responses and reduce the likelihood of hallucinations. 
For developers and data scientists, grounding usually offers the highest impact for the lowest effort.

Training a Large Language Model has a high computational cost. 
According to Wikipedia, OpenAI's GPT-3 model was link:https://en.wikipedia.org/wiki/GPT-3[trained on 175 billion parameters^], and the resulting trained model takes 800GB to store.

Retraining a model on new data would be expensive and time-consuming. 
A model may take weeks or months to train.

For exa

In [ ]:
with driver.session(database="neo4j") as session:
    response = session.run("MATCH (:Paragraph {text: $last_page})-[:MENTIONS]->(t:Topic) RETURN count(t)", {"last_page": last_page})

    for record in response:
        print(record)

<Record count(t)=30>


<ipython-input-50-f729d14f7634>:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database="neo4j") as session:


In [ ]:
graph = Neo4jGraph(
    url=neo_url,
    username=neo_user,
    password=neo_pass,
)


In [ ]:

resp = genai.embed_content(
    model="models/text-embedding-004",
    content="What is hallucination?"
)

embedding = resp["embedding"]
print(type(embedding))

<class 'list'>


In [ ]:
resp = graph.query("""
CALL db.index.vector.queryNodes('paragraphs', 6, $embedding)
YIELD node, score
RETURN node.text, score
""", {"embedding": embedding})

print(resp)

[{'node.text': '= Avoiding Hallucination\n:order: 2\n:type: lesson\n\nAs you learned in the previous lesson, LLMs can "make things up".\n\nLLMs are designed to generate human-like text based on the patterns they\'ve identified in vast amounts of data. \n\nDue to their reliance on patterns and the sheer volume of training information, LLMs sometimes **hallucinate** or produce outputs that manifest as generating untrue facts, asserting details with unwarranted confidence, or crafting plausible yet nonsensical explanations.\n\nThese manifestations arise from a mix of _overfitting_, biases in the training data, and the model\'s attempt to generalize from vast amounts of information.\n\n== Common Hallucination Problems\n\nLet\'s take a closer look at some reasons why this may occur.\n\n=== Temperature\n\nLLMs have a _temperature_, corresponding to the amount of randomness the underlying model should use when generating the text.\n\nThe higher the temperature value, the more random the gener

In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(
    url=neo_url,
    username=neo_user,
    password=neo_pass,
)

cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)


In [ ]:
print(graph.schema)

Node properties:
Course {name: STRING}
Module {name: STRING}
Lesson {name: STRING, url: STRING}
Paragraph {text: STRING, embedding: LIST}
Topic {name: STRING}
Relationship properties:

The relationships:
(:Course)-[:HAS_MODULE]->(:Module)
(:Module)-[:HAS_LESSON]->(:Lesson)
(:Lesson)-[:CONTAINS]->(:Paragraph)
(:Paragraph)-[:MENTIONS]->(:Topic)


In [ ]:
query = "How many Lessons contain paragraphs that mention the topic vector index"
result = cypher_chain.invoke({"query": query})
print(result['result'])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (l:Lesson)-[:CONTAINS]->(p:Paragraph)-[:MENTIONS]->(t:Topic {name: "vector index"})
RETURN count(DISTINCT l)

Full Context:
[{'count(DISTINCT l)': 3}]

> Finished chain.
There are 3 lessons that contain paragraphs mentioning the topic vector index. 



In [ ]:
query = "What is average number topics each paragraph mentions"
result = cypher_chain.invoke({"query": query})
print(result['result'])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Paragraph)-[:MENTIONS]->(t:Topic)
WITH p, count(t) AS topic_count
RETURN avg(topic_count)

Full Context:
[{'avg(topic_count)': 27.89887640449438}]

> Finished chain.
The average number of topics each paragraph mentions is 27.89887640449438. 

